[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/odoo_bc_rev_location.ipynb)

In [0]:
# -*- coding: utf-8 -*-
##############################################################################
#
#    OpenERP, Open Source Management Solution
#
##############################################################################
import time
import os
import base64
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from openerp.osv import fields, osv
from openerp.tools.translate import _
from openerp.tools import DEFAULT_SERVER_DATE_FORMAT, DEFAULT_SERVER_DATETIME_FORMAT
from dateutil import parser

class report_online_performance(osv.osv_memory):
    
    _name = "report.online.performance"
    _columns = {
        'date_start': fields.date('Date From', required=True),
        'date_end': fields.date('Date To', required=True),
        'file_output':fields.binary(string="File Output",invisible=True, readonly=True, help='Output file in xlsx format'),
        'file_name':fields.char(string='File Name', invisible=True)
    }
    _defaults = {
         'date_start': time.strftime('%Y-%m-01'),
         'date_end': lambda *a: str(datetime.now() + relativedelta(months=+1, day=1, days=-1))[:10],
    }
    
#     def print_report(self, cr, uid, ids, context=None):
#         if context is None:
#             context = {}
#         datas = {'ids': context.get('active_ids', [])}
#         res = self.read(cr, uid, ids, ['date_start', 'date_end'], context=context)
#         res = res and res[0] or {}
#         datas['form'] = res
#         if res.get('id',False):
#             datas['ids']=[res['id']]
#         return {'type': 'ir.actions.report.xml', 'report_name': 'bc_doanh_thu_theo_location' , 'datas': datas}
    
    def print_excel(self, cr, uid, ids, context=None):
        #set date to query_data
        #============================================================================================#        
        
        def first_day_of_month(any_day):
            any_day = parser.parse(any_day)
            return (any_day.replace(day=1)).strftime("%Y-%m-%d %H:%M:%S")

        def last_day_of_month(any_day):
            any_day = parser.parse(any_day)
            next_month = any_day.replace(day=28, hour=11, minute=59, second=59) + timedelta(days=4)
            return (next_month - timedelta(days=next_month.day)).strftime("%Y-%m-%d %H:%M:%S")
        
        datas = {'ids': context.get('active_ids', [])}
        date_start = self.read(cr, uid, ids, ['date_start'], context=context)
        date_start = first_day_of_month(date_start)
        date_end   = self.read(cr, uid, ids, ['date_end'], context=context)
        date_end   = last_day_of_month(date_end)
        
        out_path = '/tmp/bc_ton_size_%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        engine = pg.connect('postgresql://odoo:odoo@***:***/***')
        #Run report
        #============================================================================================# 
        def get_name():
            get_location_id ='''
                SELECT id, name
                FROM 
                    stock_warehouse
            '''
            df = pd.read_sql(get_location_id,engine)
            df['name'] = df['name'].apply(lambda x: x.replace('KHO', 'CH'))
            return df.set_index('id').to_dict()['name']

        def get_revenue(date_start,date_end, pkt=False):

            #check revenue from voucher or product. If True, it's meaning revenue from voucher
            if pkt:
                condition = 'like'
                name = 'voucher_rev'
            else:
                condition = 'not like'
                name = 'product_rev'

            #query to get data from database
            get_revenue ='''
            select tmp.id, tmp.month, sum(tmp.rev) as %s
            from
                (select s.warehouse_id as id,date_part('month', s.date_order)as month, sum(product_uom_qty * price_unit) as rev
                from sale_order_line l
                      join sale_order s on (l.order_id=s.id)
                    left join product_product p on (l.product_id=p.id)
                        left join product_template t on (p.product_tmpl_id=t.id)
                where s.state not in ('draft','cancel') 
                    AND s.date_order between '%s' and '%s'
                    AND t.name %s '%sPQT%s'
                Group by 1, 2

                union

                select w.id,date_part('month', s.date_order) as month, sum(price_subtotal) as rev
                from pos_order_line l
                    join pos_order s on (l.order_id=s.id)
                    left join stock_warehouse w on (s.location_id = w.lot_stock_id)
                    left join product_product p on (l.product_id=p.id)
                        left join product_template t on (p.product_tmpl_id=t.id)
                where s.state not in ('draft','cancel') 
                    AND s.date_order between '%s' and '%s'
                    AND t.name %s '%sPQT%s'
                Group by 1, 2) tmp
            Group by 1, 2
            '''%(name, date_start, date_end, condition, '%','%', date_start, date_end, condition, '%','%')

            #get data from database and convert to dataframe
            df = pd.read_sql(get_revenue,engine)
            df['month'] = df['month'].astype('int')
        #     df['new_id'] = df['id'].apply(str) + '-' + df['month'].apply(str)
        #     df = df.set_index('new_id')

            return df

        def get_paid_by_voucher(date_start, date_end):

            #query to get total amount revenue had been paid by voucher
            get_paid_by_voucher ='''
                select w.id, date_part('month', p.date_order) as month, sum(a.amount) paid_by_voucher
                from account_bank_statement_line a
                    join account_journal j on (a.journal_id = j.id)
                    join pos_order p on (a.pos_statement_id = p.id)
                    left join stock_warehouse w on (p.location_id = w.lot_stock_id)
                where p.state not in ('draft','cancel') 
                    AND p.date_order between '%s' and '%s'
                    AND j.code like '%sPQT%s'
                group by 1,2
            '''%(date_start, date_end,'%','%')

            #get data from database and convert to dataframe
            df = pd.read_sql(get_paid_by_voucher,engine)
            df['month'] = df['month'].astype('int')

            return df
       
        def crate_df(date_start,date_end):

            #get necessary data from database
            product_revenue = get_revenue(date_start,date_end)
            voucher_revenue = get_revenue(date_start,date_end, pkt=True)
            paid_by_voucher = get_paid_by_voucher(date_start,date_end)
            location_name = get_name()

            #merge data
            result = pd.merge(product_revenue, voucher_revenue, how='outer', on=['id', 'month'])
            result = pd.merge(result, paid_by_voucher, how='outer', on=['id', 'month'])

            #convert data form
            final = pd.DataFrame(data={'id': [i for i in set(result.id)]})
            for month in set(result.month):
                month_df = result[result['month']==month].drop(['month'], axis=1)
                final = pd.merge(final, month_df, how='outer', on=['id'])

            #clean data
            final = final[final['id'] != 2]
            final = final.fillna(0)
            final['id'] = final['id'].map(location_name)
            #final = final.apply(lambda x: '{:d}'.format(x))

            return final, set(result.month)
        
        def create_excel(date_start, date_end):
            #get data
            final, set_month = crate_df(date_start, date_end)
            location_name = get_name()

            #create excel file
            writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
            final.to_excel(writer,sheet_name = 'Final_report', startrow=4, startcol=0, index=False)

            #set workbook's infor
            workbook = writer.book
            worksheet = writer.sheets['Final_report']
            header_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#B7DEE8',  'border':1, 'align' :'center'})
            big_size = workbook.add_format({'bold': True, 'font_size':15})

            #write workbook
            worksheet.write(1, 1, 'Báo Cáo Doanh Thu Theo Cửa Hàng', big_size)
            worksheet.write(4, 0, 'Location')
            worksheet.set_column('A:%s'%(chr(65 + 3*len(set_month))), 23)

            for i in range(len(set_month)):
                worksheet.merge_range(3, i*3+1, 3, i*3+3, "Tháng %s"%(list(set_month)[i]), header_format)
                worksheet.write(4, i*3+1, 'Doanh thu', header_format)
                worksheet.write(4, i*3+2, 'Bán PQT', header_format)
                worksheet.write(4, i*3+3, 'Thu PQT', header_format)

            writer.save()
        
        #run report 
        create(date_start, date_end)
        
        #Encode final file to binary and download it.
        #============================================================================================#
        
        PREVIEW_PATH = out_path
        encoded_string = ""     
        with open(PREVIEW_PATH, "r") as image_file:
            encoded_string = base64.encodestring(image_file.read())#.b64encode
        for order in self.browse(cr, uid, ids[0]):
            order.file_name = '%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            order.file_output = encoded_string
            data_id =  order.id
         
        return {
            'type': 'ir.actions.act_window',
            'res_model': 'report.online.performance',
            'view_mode': 'form',
            'view_type': 'form',
            'res_id': data_id,
            'views': [(False, 'form')],
            'target': 'new',
        }

# vim:expandtab:smartindent:tabstop=4:softtabstop=4:shiftwidth=4:
